Dans ce notebook, nous allons essayer de faire du text mining pour récuper des versions locales des programmes des présidentielles 2017 des candidats suivants :

- François Fillon (apparemment, le projet sort le 13 mars seulement)
- Marine Le Pen
- Benoît Hamon
- Jean-Luc Mélenchon
- Emmanuel Macron

Différentes manières de récupérer les programmes sont possibles : soit à partir des fichiers .pdf, soit à partir des sites des campagnes. Nous allons faire au plus simple, à l'aide des sites de campagnes.

# François Fillon 

Le projet de François Fillon ne sera annoncé que le 13 mars : https://www.fillon2017.fr/projet/

# Marine Le Pen 

Les 144 engagements de Marine Le Pen peuvent être consultés ici : https://www.marine2017.fr/programme/

## Analyse de la structure du site

Apparemment, les différentes propositions sont imbriquées dans des balises `<p>`. 

```
<p>3. <strong>Permettre la représentation de tous les Français</strong> par le scrutin proportionnel à toutes les élections. À l’Assemblée nationale, la proportionnelle sera intégrale avec une prime majoritaire de 30&nbsp;% des sièges pour la liste arrivée en tête et un seuil de 5&nbsp;% des suffrages pour obtenir des élus.</p>
```
On peut donc extraire ces éléments et les trier ensuite.

## Extraction des paragraphes

Téléchargeons le code source de la page.

In [53]:
from bs4 import BeautifulSoup
import requests
import re

In [54]:
r = requests.get('https://www.marine2017.fr/programme/')

In [55]:
soup = BeautifulSoup(r.text, "html.parser")

Maintenant, chercons à extraire tous les paragraphes, à l'aide d'une fonction qui vérifie que le paragraphe commence par un nombre suivi d'un point (et peut-être d'un espace).

In [56]:
def func(tag):
    "Filters tag that start with a number."
    pattern = re.compile('\d+.\s*')
    return pattern.match(tag.text) is not None

In [57]:
all_paragraphs = soup.find_all(func)

Explorons le résultat.

In [59]:
from ipywidgets import interact

In [60]:
@interact
def browser_paragraphs(n=(0, len(all_paragraphs) - 1)):
    print(all_paragraphs[n].text)

Il semblerait que seul le premier paragraphe soit repris deux fois. Ce n'est pas grave, on peut facilement faire le tri et obtenir une table des 144 propositions.

In [61]:
import pandas as pd

In [74]:
s = pd.Series([p.text[3:].strip() for p in all_paragraphs]).drop_duplicates().reset_index(drop=True)
df = pd.DataFrame(data=s, columns=['texte de la proposition'])
df.head(10)

,texte de la proposition
0,Retrouver notre liberté et la maîtrise de notr...
1,Organiser un référendum en vue de réviser la C...
2,Permettre la représentation de tous les França...
3,Abaisser le nombre de députés à 300 (contre 57...
4,Créer un véritable référendum d’initiative pop...
5,Conserver trois niveaux d’administration (au l...
6,Garantir la liberté d’expression et les libert...
7,Créer une charte à valeur constitutionnelle qu...
8,Défendre les droits des femmes : lutter contre...
9,Assurer le respect de la liberté d’association...


Bien, on peut maintenant écrire ces données dans un fichier texte.

In [75]:
df.to_csv('projets/marine_le_pen.csv')

# Benoît Hamon 

Le site de Benoît Hamon ne permet pas d'accéder à une page avec toutes les propositions facilement. Du coup, il faut explorer trois sous-catégories.

https://www.benoithamon2017.fr/thematique/pour-un-progres-social-et-ecologique/

In [77]:
r = requests.get('https://www.benoithamon2017.fr/thematique/pour-un-progres-social-et-ecologique/')
r

<Response [200]>

In [80]:
soup = BeautifulSoup(r.text, 'html.parser')

In [86]:
all_propositions = soup.find_all(class_='Propositions-Proposition')

In [87]:
len(all_propositions)

66

In [88]:
p = all_propositions[0]

In [90]:
p.text

'\nLutte contre les déserts médicaux\nJe lutterai contre les déserts médicaux en retirant le conventionnement aux médecins qui s’installent en zone surdotée. Cette mesure n’entrave en rien la liberté d’installation du médecin, qui peut toujours choisir de s’y installer\xa0: il ne bénéficiera simplement plus du conventionnement. J’encouragerai le développement de maisons de santé pluridisciplinaires rassemblant des médecins libéraux, appuyés pour les tâches administratives par un gestionnaire des fonctions support. \n\n\nJe soutiens, je partage sur les réseaux #DésertsMédicaux\n\n\n\nEn savoir plus\n'

In [92]:
p.find('h1').text

'Lutte contre les déserts médicaux'

In [93]:
p.find('p').text

'Je lutterai contre les déserts médicaux en retirant le conventionnement aux médecins qui s’installent en zone surdotée. Cette mesure n’entrave en rien la liberté d’installation du médecin, qui peut toujours choisir de s’y installer\xa0: il ne bénéficiera simplement plus du conventionnement. J’encouragerai le développement de maisons de santé pluridisciplinaires rassemblant des médecins libéraux, appuyés pour les tâches administratives par un gestionnaire des fonctions support.'

On peut extraire de ces propositions la moëlle essentielle :

In [97]:
def extract_data(tag):
    "Extracts title for tag and content."
    subject = tag.find('h1').text
    content = tag.find('p').text
    return subject, content

Construisons une table de données avec ces propositions.

In [102]:
df = pd.DataFrame([extract_data(p) for p in all_propositions], columns=['titre', 'contenu'])

In [103]:
df

,titre,contenu
0,Lutte contre les déserts médicaux,Je lutterai contre les déserts médicaux en ret...
1,Abrogation de la loi Travail,J’abrogerai immédiatement la loi Travail. Je r...
2,Un budget de 1% du PIB dédié à la culture,Je porterai le budget consacré au développemen...
3,Création d’un Revenu universel d’existence,Je mettrai en place un revenu universel d’exis...
4,Mise en place d’un statut social unique de l’a...,Je créerai un statut unique pour tous les acti...
5,"Oui au travail indépendant, non à l’ubérisatio...",Je lutterai contre le salariat déguisé des ent...
6,Plan d’investissements dans la rénovation éner...,Je lancerai un plan massif d’investissements d...
7,Annulation de la dette contractée par les pays...,J’organiserai avec nos partenaires européens l...
8,Création d’une taxe sur les robots,Je créerai une taxe sur la richesse créée par ...
9,Reconnaissance d’un statut de l’artiste,Je donnerai corps à un statut de l’artiste afi...


Faisons la même chose avec les autres sous-pages.

In [104]:
dfs = []
for url in ['https://www.benoithamon2017.fr/thematique/pour-un-progres-social-et-ecologique/',
            'https://www.benoithamon2017.fr/thematique/pour-une-republique-bienveillante-et-humaniste/',
            'https://www.benoithamon2017.fr/thematique/pour-une-france-independante-et-protectrice/']:
    r = requests.get('https://www.benoithamon2017.fr/thematique/pour-un-progres-social-et-ecologique/')
    soup = BeautifulSoup(r.text, 'html.parser')
    all_propositions = soup.find_all(class_='Propositions-Proposition')
    print('url: {}, nombre de propositions: {}'.format(url, len(all_propositions)))
    df = pd.DataFrame([extract_data(p) for p in all_propositions], columns=['titre', 'contenu'])
    dfs.append(df)

url: https://www.benoithamon2017.fr/thematique/pour-un-progres-social-et-ecologique/, nombre de propositions: 66
url: https://www.benoithamon2017.fr/thematique/pour-une-republique-bienveillante-et-humaniste/, nombre de propositions: 66
url: https://www.benoithamon2017.fr/thematique/pour-une-france-independante-et-protectrice/, nombre de propositions: 66


In [109]:
df = pd.concat(dfs).drop_duplicates().reset_index(drop=True)
df

,titre,contenu
0,Lutte contre les déserts médicaux,Je lutterai contre les déserts médicaux en ret...
1,Abrogation de la loi Travail,J’abrogerai immédiatement la loi Travail. Je r...
2,Un budget de 1% du PIB dédié à la culture,Je porterai le budget consacré au développemen...
3,Création d’un Revenu universel d’existence,Je mettrai en place un revenu universel d’exis...
4,Mise en place d’un statut social unique de l’a...,Je créerai un statut unique pour tous les acti...
5,"Oui au travail indépendant, non à l’ubérisatio...",Je lutterai contre le salariat déguisé des ent...
6,Plan d’investissements dans la rénovation éner...,Je lancerai un plan massif d’investissements d...
7,Annulation de la dette contractée par les pays...,J’organiserai avec nos partenaires européens l...
8,Création d’une taxe sur les robots,Je créerai une taxe sur la richesse créée par ...
9,Reconnaissance d’un statut de l’artiste,Je donnerai corps à un statut de l’artiste afi...


In [110]:
df.to_csv('projets/benoit_hamon.csv')

# Jean-Luc Mélenchon 

On peut trouver une version inofficielle du programme ici : https://laec.fr/sommaire

Un peu comme pour le site d'Hamon, il y a des rubriques. Commençons par la première.

In [361]:
r = requests.get('https://laec.fr/chapitre/1/la-6e-republique')

In [362]:
soup = BeautifulSoup(r.text, 'html.parser')

In [363]:
sublinks = soup.find_all('a', class_='list-group-item')

In [364]:
sublinks

[<a class="list-group-item toc-section toc-chapter-la-6e-republique" href="/section/1/reunir-une-assemblee-constituante"><span class="label label-default">1</span> Réunir une Assemblée constituante </a>,
 <a class="list-group-item toc-section toc-chapter-la-6e-republique" href="/section/2/balayer-l-oligarchie-abolir-les-privileges-de-la-caste"><span class="label label-default">2</span> Balayer l'oligarchie, abolir les privilèges de la caste </a>,
 <a class="list-group-item toc-section toc-chapter-la-6e-republique" href="/section/3/une-republique-permettant-l-intervention-populaire"><span class="label label-default">3</span> Une République permettant l'intervention populaire </a>,
 <a class="list-group-item toc-section toc-chapter-la-6e-republique" href="/section/4/abolir-la-monarchie-presidentielle"><span class="label label-default">4</span> Abolir la monarchie présidentielle </a>,
 <a class="list-group-item toc-section toc-chapter-la-6e-republique" href="/section/5/une-nouvelle-etape-

On peut étendre cette manière de récupérer les données à toutes les sous-sections :

In [365]:
suburls = ['https://laec.fr/chapitre/1/la-6e-republique', 
           'https://laec.fr/chapitre/2/proteger-et-partager',
           'https://laec.fr/chapitre/3/la-planification-ecologique',
           'https://laec.fr/chapitre/4/sortir-des-traites-europeens',
           'https://laec.fr/chapitre/5/pour-l-independance-de-la-france',
           'https://laec.fr/chapitre/6/le-progres-humain-d-abord',
           'https://laec.fr/chapitre/7/la-france-aux-frontieres-de-l-humanite']

In [366]:
sublinks = []
for suburl in suburls:
    r = requests.get(suburl)
    soup = BeautifulSoup(r.text, 'html.parser')
    sublinks.extend(soup.find_all('a', class_='list-group-item'))

In [367]:
sublinks[:10]

[<a class="list-group-item toc-section toc-chapter-la-6e-republique" href="/section/1/reunir-une-assemblee-constituante"><span class="label label-default">1</span> Réunir une Assemblée constituante </a>,
 <a class="list-group-item toc-section toc-chapter-la-6e-republique" href="/section/2/balayer-l-oligarchie-abolir-les-privileges-de-la-caste"><span class="label label-default">2</span> Balayer l'oligarchie, abolir les privilèges de la caste </a>,
 <a class="list-group-item toc-section toc-chapter-la-6e-republique" href="/section/3/une-republique-permettant-l-intervention-populaire"><span class="label label-default">3</span> Une République permettant l'intervention populaire </a>,
 <a class="list-group-item toc-section toc-chapter-la-6e-republique" href="/section/4/abolir-la-monarchie-presidentielle"><span class="label label-default">4</span> Abolir la monarchie présidentielle </a>,
 <a class="list-group-item toc-section toc-chapter-la-6e-republique" href="/section/5/une-nouvelle-etape-

Combien de propositions trouvons-nous ?

In [368]:
len(sublinks)

83

Maintenant regardons comment extraire les informations.

In [369]:
link = sublinks[0]

In [370]:
link

<a class="list-group-item toc-section toc-chapter-la-6e-republique" href="/section/1/reunir-une-assemblee-constituante"><span class="label label-default">1</span> Réunir une Assemblée constituante </a>

In [371]:
full_url = 'https://laec.fr' + link.attrs['href']
full_url

'https://laec.fr/section/1/reunir-une-assemblee-constituante'

In [372]:
r = requests.get(full_url)

In [373]:
r.text

'<!DOCTYPE html>\n<html lang="fr" class="page no-js not-logged-in theme-frontend theme-chapter-la-6e-republique page-section page-section-1">\n<!--\n\n\n░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░\n░░░█░░░█▀█░█▀▀░█▀▀░░░░█▀▀░█▀▄░░░░\n░░░█░░░█▀█░█▀▀░█░░░░░░█▀▀░█▀▄░░░░\n░░░▀▀▀░▀░▀░▀▀▀░▀▀▀░▀░░▀░░░▀░▀░░░░\n░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░\n+===== L\'Avenir En Commun ======+\n\n-->\n  <head>\n    <!-- Classe .js/.no-js https://www.paulirish.com/2009/avoiding-the-fouc-v3/ -->\n    <script>(function(H){H.className=H.className.replace(/\\bno-js\\b/,\'js\')})(document.documentElement)</script>\n    \n    <meta charset="UTF-8">\n    <meta name="viewport" content="width=device-width, initial-scale=1, shrink-to-fit=no">\n\n    <title>Réunir une Assemblée constituante</title>\n    <meta name="description" content="La nouvelle Constitution dont la France a besoin doit être radicalement nouvelle, y compris dans sa méthode d&#039;écriture\xa0: elle ne peut être un simple rafistolage de la 5e République,…"/>\n\n

In [374]:
soup = BeautifulSoup(r.text, 'html.parser')

In [375]:
soup.find(class_='number').text

'1'

In [376]:
soup.find(class_='text-balance').text

'Réunir une Assemblée constituante'

In [377]:
soup.find(class_='semi-lead subject-foreword').text

"La nouvelle Constitution dont la France a besoin doit être radicalement nouvelle, y compris dans sa méthode d'écriture\xa0: elle ne peut être un simple rafistolage de la 5e République, ni se résumer à quelques réformes octroyées par le futur président de la République. C'est le peuple lui-même qui doit s'emparer de la question et s'impliquer tout au long d'un processus constituant. Nous proposons la convocation d'une assemblée spécifiquement chargée de rédiger une nouvelle Constitution sous le contrôle des citoyens\xa0: une Assemblée constituante. Nous soumettrons à ses travaux des propositions pour une 6e République démocratique, égalitaire, instituant de nouveaux droits et imposant l'impératif écologique."

In [380]:
soup.find(class_='list-measures').find_all('li')

[<li class="list-group-item" data-measure-seq="1" data-target="#modal-measure-1" id="mesure-1"><i aria-hidden="true" class="fa fa-pull-right fa-heart icon-bookmarked hidden" title="Dans vos favoris !"></i><p>Convoquer un référendum (article 11 de la Constitution) pour engager le processus constituant (modalités de la composition de l'Assemblée constituante — mode de scrutin, parité, tirage au sort et incompatibilités ; modalités de la délibération ; association des citoyens aux travaux…)</p></li>,
 <li class="list-group-item" data-measure-seq="2" data-target="#modal-measure-2" id="mesure-2"><i aria-hidden="true" class="fa fa-pull-right fa-heart icon-bookmarked hidden" title="Dans vos favoris !"></i><p>Aucun parlementaire des anciennes assemblées de la 5e République ne pourra siéger dans cette Assemblée constituante. Les délégués à l'Assemblée constituante ne pourront être candidats aux élections suivant l'entrée en vigueur de la nouvelle Constitution</p></li>,
 <li class="list-group-it

In [382]:
[_.text for _ in soup.find(class_='list-measures').find_all('li')]


["Convoquer un référendum (article 11 de la Constitution) pour engager le processus constituant (modalités de la composition de l'Assemblée constituante\xa0— mode de scrutin, parité, tirage au sort et incompatibilités\xa0; modalités de la délibération\xa0; association des citoyens aux travaux…)",
 "Aucun parlementaire des anciennes assemblées de la 5e République ne pourra siéger dans cette Assemblée constituante. Les délégués à l'Assemblée constituante ne pourront être candidats aux élections suivant l'entrée en vigueur de la nouvelle Constitution",
 "Le projet de Constitution proposé par l'Assemblée constituante sera soumis à un référendum d'approbation"]

In [390]:
def extract_data(link):
    full_url = 'https://laec.fr' + link.attrs['href']
    r = requests.get(full_url)
    soup = BeautifulSoup(r.text, 'html.parser')
    number = soup.find(class_='number').text
    title = soup.find(class_='text-balance').text
    if soup.find(class_='semi-lead subject-foreword') is not None:
        foreword = soup.find(class_='semi-lead subject-foreword').text
    else:
        foreword = ''
    if soup.find(class_='list-measures') is not None:
        content = ". ".join([_.text for _ in soup.find(class_='list-measures').find_all('li')]) + "."
    else:
        content = ''
    return number, title, foreword, content

In [391]:
extract_data(link)

('1',
 'Réunir une Assemblée constituante',
 "La nouvelle Constitution dont la France a besoin doit être radicalement nouvelle, y compris dans sa méthode d'écriture\xa0: elle ne peut être un simple rafistolage de la 5e République, ni se résumer à quelques réformes octroyées par le futur président de la République. C'est le peuple lui-même qui doit s'emparer de la question et s'impliquer tout au long d'un processus constituant. Nous proposons la convocation d'une assemblée spécifiquement chargée de rédiger une nouvelle Constitution sous le contrôle des citoyens\xa0: une Assemblée constituante. Nous soumettrons à ses travaux des propositions pour une 6e République démocratique, égalitaire, instituant de nouveaux droits et imposant l'impératif écologique.",
 "Convoquer un référendum (article 11 de la Constitution) pour engager le processus constituant (modalités de la composition de l'Assemblée constituante\xa0— mode de scrutin, parité, tirage au sort et incompatibilités\xa0; modalités de

In [392]:
raw_data = [extract_data(link) for link in sublinks]

In [393]:
df = pd.DataFrame(raw_data, columns=['numéro', 'titre', 'introduction', 'mesures'])
df

,numéro,titre,introduction,mesures
0,1,Réunir une Assemblée constituante,La nouvelle Constitution dont la France a beso...,Convoquer un référendum (article 11 de la Cons...
1,2,"Balayer l'oligarchie, abolir les privilèges de...",La démocratie française est malade des privilè...,Rendre inéligible à vie toute personne condamn...
2,3,Une République permettant l'intervention popul...,Le peuple est systématiquement tenu à l'écart ...,"Fixer le droit de vote à 16 ans, instaurer le ..."
3,4,Abolir la monarchie présidentielle,La monarchie présidentielle actuelle marginali...,Abolir la monarchie présidentielle en instaura...
4,5,Une nouvelle étape des libertés et de l'émanci...,"Une Constitution, c'est la garantie pour le pe...",Constitutionnaliser la non-marchandisation du ...
5,6,Une République universelle,Aucune liberté n'est possible sans l'égalité e...,Faire France de tout boisDéfendre le droit du ...
6,7,Une République laïque,La laïcité est attaquée de toutes parts et ins...,Étendre le bénéfice de l'application de la loi...
7,8,La révolution citoyenne dans les médias,Il n'y a pas de démocratie possible sans infor...,Faire élire les présidents de France Télévisio...
8,9,La République garante des biens communs,La République est un mot creux si elle ignore ...,Constitutionnaliser la règle verte : ne pas pr...
9,10,Reconnaître la citoyenneté dans l'entreprise e...,"Comme le disait Jean Jaurès, « la grande Révol...",Accorder de nouveaux droits de contrôle aux co...


On écrit un fichier.

In [394]:
df.to_csv('projets/jean_luc_melenchon.csv')

# Emmanuel Macron 

Il faut dans un premier temps aller chercher les pages individuelles du site.

In [268]:
r = requests.get('https://en-marche.fr/emmanuel-macron/le-programme')
soup = BeautifulSoup(r.text, 'html.parser')

In [269]:
proposals = soup.find_all(class_='programme__proposal')
proposals = [p for p in proposals if 'programme__proposal--category' not in p.attrs['class']]

In [270]:
len(proposals)

36

In [271]:
full_urls = ["https://en-marche.fr" + p.find('a').attrs['href'] for p in proposals]

In [309]:
url = full_urls[1]

In [310]:
r = requests.get(url)

In [311]:
text = r.text

In [312]:
text = text.replace('</br>', '')

In [315]:
soup = BeautifulSoup(text, 'html.parser')

In [316]:
article_tag = soup.find_all('article', class_='l__wrapper--slim')[0]

In [317]:
for line in article_tag.find_all(class_='arrows'):
    print(line.text) 


Nous agirons pour que les agriculteurs pèsent plus dans leurs négociations avec les industriels de l’agro-alimentaire.


Nous encouragerons le développement de véritables organisations de producteurs, avec des capacités de négociations renforcées pour peser plus dans les négociations commerciales avec les centrales d’achat de la grande distribution.


Nous organiserons un Grenelle de l’alimentation avec les représentants des agriculteurs, des industries de transformation, de la distribution et des consommateurs, afin de définir un partage équilibré de la valeur.


Nous protégerons les agriculteurs contre la volatilité des prix par la mise en place d’outils de régulation adaptés à chaque filière.


Nous proposerons des outils de gestion des risques efficaces et adaptés.


Nous permettrons aux agriculteurs de conserver les mêmes règles du jeu pour être compétitifs : favoriser la convergence sociale et fiscale au niveau européen.


Nous créerons un droit à l’erreur pour tous.


Nous donn

In [325]:
tag = article_tag.find_all(class_='arrows')[-1]

In [326]:
tag.text

'\nNous ferons confiance aux territoires pour s’organiser et trouver des solutions adaptées.\n'

In [328]:
tag.next_sibling

'\nNous encouragerons les projets alimentaires territoriaux (PAT) pour rapprocher les producteurs, les transformateurs, les distributeurs, les collectivités territoriales et atteindre 50% de produits biologiques, écologiques ou locaux dans l’ensemble de la restauration collective en 2022. \n'

In [306]:
def extract_items(url):
    r = requests.get(url)
    text = r.text.replace('</br>', '')
    soup = BeautifulSoup(text, 'html.parser')
    article_tag = soup.find_all('article', class_='l__wrapper--slim')[0]
    return [line.text.strip() for line in article_tag.find_all(class_='arrows')]

In [308]:
extract_items(full_urls[1])

['Nous agirons pour que les agriculteurs pèsent plus dans leurs négociations avec les industriels de l’agro-alimentaire.',
 'Nous encouragerons le développement de véritables organisations de producteurs, avec des capacités de négociations renforcées pour peser plus dans les négociations commerciales avec les centrales d’achat de la grande distribution.',
 'Nous organiserons un Grenelle de l’alimentation avec les représentants des agriculteurs, des industries de transformation, de la distribution et des consommateurs, afin de définir un partage équilibré de la valeur.',
 'Nous protégerons les agriculteurs contre la volatilité des prix par la mise en place d’outils de régulation adaptés à chaque filière.',
 'Nous proposerons des outils de gestion des risques efficaces et adaptés.',
 'Nous permettrons aux agriculteurs de conserver les mêmes règles du jeu pour être compétitifs : favoriser la convergence sociale et fiscale au niveau européen.',
 'Nous créerons un droit à l’erreur pour tous

On extrait toutes les propositions.

In [336]:
propositions = [extract_items(url) for url in full_urls]

In [348]:
len(propositions)

36

In [350]:
full_urls[18]

'https://en-marche.fr/emmanuel-macron/le-programme/international'

In [349]:
@interact
def print_prop(n=(0, len(propositions) - 1)):
    print(propositions[n])

In [342]:
propositions[0] + propositions[1]

['Nous élargirons les horaires d’ouverture des services publics pour les adapter aux contraintes des usagers : certains services publics ouvriront le samedi et en soirée, afin d’être plus facilement accessibles par tous les usagers, et notamment ceux qui travaillent toute la semaine et en journée.',
 'D’ici 2022, 100% des démarches administratives pourront être effectuées depuis Internet – sauf première délivrance des documents d’identité officiels.',
 'Nous créerons un compte citoyen en ligne (site et application), qui rassemblera sur une même interface tous les droits, notamment ceux liés à la santé, à la trajectoire professionnelle, à la formation, à la situation fiscale, aux droits civiques.',
 'D’ici la fin 2017, les directeurs d’administration centrale et les principaux dirigeants des organismes publics seront nommés ou confirmés au terme d’une procédure transparente, après revue de compétences et entretien.',
 'S’ils décident de quitter le service de l’État, les hauts fonctionna

In [351]:
sum([prop for prop in propositions if prop is not None])

TypeError: unsupported operand type(s) for +: 'int' and 'list'

In [ ]:
sum()

In [354]:
all_props = []
for props in propositions:
    if props is not None:
        all_props = all_props + props

In [355]:
all_props

['Nous élargirons les horaires d’ouverture des services publics pour les adapter aux contraintes des usagers : certains services publics ouvriront le samedi et en soirée, afin d’être plus facilement accessibles par tous les usagers, et notamment ceux qui travaillent toute la semaine et en journée.',
 'D’ici 2022, 100% des démarches administratives pourront être effectuées depuis Internet – sauf première délivrance des documents d’identité officiels.',
 'Nous créerons un compte citoyen en ligne (site et application), qui rassemblera sur une même interface tous les droits, notamment ceux liés à la santé, à la trajectoire professionnelle, à la formation, à la situation fiscale, aux droits civiques.',
 'D’ici la fin 2017, les directeurs d’administration centrale et les principaux dirigeants des organismes publics seront nommés ou confirmés au terme d’une procédure transparente, après revue de compétences et entretien.',
 'S’ils décident de quitter le service de l’État, les hauts fonctionna

In [359]:
df = pd.DataFrame(all_props, columns=['proposition'])
df

,proposition
0,Nous élargirons les horaires d’ouverture des s...
1,"D’ici 2022, 100% des démarches administratives..."
2,Nous créerons un compte citoyen en ligne (site...
3,"D’ici la fin 2017, les directeurs d’administra..."
4,S’ils décident de quitter le service de l’État...
5,"Dans la fonction publique, nous limiterons à c..."
6,Nous garantirons la transparence sur la qualit...
7,Nous créerons 10 000 postes de policiers et ge...
8,Nous réduirons le nombre d’agents publics de 1...
9,Les ministres seront comptables du respect des...


In [360]:
df.to_csv('projets/emmanuel_macron.csv')

# Yannick Jadot

http://avecjadot.fr/lafrancevive/

In [395]:
r = requests.get('http://avecjadot.fr/lafrancevive/')

In [396]:
soup = BeautifulSoup(r.text, 'html.parser')

In [399]:
tags = soup.find_all('div', class_='bloc-mesure')

In [404]:
links = [tag.find('a').attrs['href'] for tag in tags]

In [406]:
all([link.startswith('http://avecjadot.fr/') for link in links])

True

Extraction du titre d'une des pages.

In [407]:
link = links[0]

In [408]:
r = requests.get(link)

In [409]:
soup = BeautifulSoup(r.text, 'html.parser')

In [414]:
soup.find('div', class_='texte-mesure').text.strip().replace('\n', ' ')

'Adopter une loi de sortie progressive et définitive du nucléaire d’ici 2035, arrêter les premiers réacteurs dès 2017.'

In [415]:
def extract_data(link):
    r = requests.get(link)
    soup = BeautifulSoup(r.text, 'html.parser')
    return soup.find('div', class_='texte-mesure').text.strip().replace('\n', ' ')

In [416]:
extract_data(link)

'Adopter une loi de sortie progressive et définitive du nucléaire d’ici 2035, arrêter les premiers réacteurs dès 2017.'

In [418]:
all_props = [extract_data(link) for link in links]

In [419]:
df = pd.DataFrame(all_props, columns=['proposition'])
df

,proposition
0,Adopter une loi de sortie progressive et défin...
1,Les objectifs en matière d’énergies renouvelab...
2,Favoriser l’appropriation citoyenne locale de ...
3,Lutter contre le gaspillage et éradiquer les p...
4,Favoriser l’économie circulaire et lutter cont...
5,Aller vers une France zéro déchet en divisant ...
6,Développer les réseaux de transports du quotid...
7,Faire de la santé une priorité nationale. → \t...
8,Préserver notre santé en luttant contre les po...
9,Protéger notre biodiversité. → \t Vers une Fra...


In [420]:
df.to_csv('projets/yannick_jadot.csv')